-This notebook is an example of how to properly set up the ScreenBeamProfileMeasurement Class:
    -Since ScreenBeamProfileMeasurement takes a screen object as a pydantic field to use for live images and this example is mean't to be 
done offline we subclass Screen and overwrite the the image property to a static 2d array. 
        -The array passed in this example is automatically

In [ ]:
from lcls_tools.common.measurements.screen_beam_profile_measurement import ScreenBeamProfileMeasurement
from lcls_tools.common.data_analysis.projection_fit.projection_fit import ProjectionFit
from lcls_tools.common.data_analysis.projection_fit.gaussian_model import GaussianModel
from lcls_tools.common.image_processing.image_processing import ImageProcessor
from lcls_tools.common.image_processing.roi import ROI, RectangularROI, CircularROI
from lcls_tools.common.devices.device import Device, PVSet, ControlInformation, Metadata
from lcls_tools.common.devices.screen import ScreenControlInformation, ScreenPVSet, Screen
from matplotlib import pyplot as plt
from epics import PV, caget
import numpy as np
import h5py
import random

In [ ]:
class ScreenTest(Screen):
    @property
    def image(self) -> np.ndarray:
        return self._image
    
    @image.setter
    def image(self,image):
        self._image = image

In [ ]:
def create_test_image(size:tuple,center:list,radius:int):
    # make img that is a circle in the center of the image with known standard dev and mean. no imports, no calls to external or
    # internal files. 
    image = np.zeros(size)
    for y in range(image.shape[0]):
        for x in range(image.shape[1]):
            distance =  np.sqrt((x - center[0])**2 + (y - center[1])**2)
            if distance < radius:
                image[y,x]= 1
    return image

In [ ]:
# creating projection fit class
gauss_model = GaussianModel()
projection_fit = ProjectionFit(model = gauss_model,visualize_fit = True, visualize_priors = True,use_priors =True)

In [ ]:
# creating image processing class
# would be nice to have default way of finding the center of image
# also maybe have rectangularROI have default roi_type 'Rectangular', and change roi_type to roi
roi = RectangularROI(center =[400,400],xwidth=300,ywidth=300)
image_processor = ImageProcessor(roi = roi,visualize = True)

In [ ]:
PV_strings = {
    'image': 'ArrayData',
    'n_bits': 'N_OF_BITS',
    'n_col': 'ArraySize1_RBV',
    'n_row': 'ArraySize0_RBV',
    'resolution': 'RESOLUTION'
}
metadata = {
        'area': 'TEST',
        'beam_path': ['SC_TEST'],
        'sum_l_meters': 99.99
        }
control_name =  "OTRS:TEST:650:"

In [ ]:
# creating Screen Device
# inconsistent use of Controls_information and control_information between classes recommend change.
screen_pvs = ScreenPVSet(**PV_strings)
meta_data = Metadata(**metadata)
controls_information = ScreenControlInformation(control_name = control_name, PVs = screen_pvs )
screen_test = ScreenTest(controls_information = controls_information, metadata = meta_data)

In [ ]:
test = ScreenBeamProfileMeasurement(name = control_name,device = screen_test, image_processor = image_processor, fitting_tool = projection_fit)

In [ ]:
test.device.image = create_test_image((800,800),[400,400],50)

In [ ]:
test.measure()

In [ ]:
dict(test)

In [ ]:
test.fitting_tool.plot_fit()